In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from keras.preprocessing.text import Tokenizer


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
embeddings_index = dict()
f = open('/content/drive/My Drive/glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
docs=embeddings_index.keys()
tokenizer_en = Tokenizer(num_words=400000)
tokenizer_en.fit_on_texts(docs)
vocab_size = len(tokenizer_en.word_index) + 3
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer_en.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
train_dataset_en=tf.data.TextLineDataset('/content/drive/My Drive/DEEP LEARNING ASSIGNMENTS/A4_set2/train_en.txt')
train_dataset_ta=tf.data.TextLineDataset('/content/drive/My Drive/DEEP LEARNING ASSIGNMENTS/A4_set2/train_ta.txt')
train_dataset_en=[str(i.decode('utf-8')).replace('\'',' \'') for i in train_dataset_en.as_numpy_iterator()]
train_dataset_en=tf.data.Dataset.from_tensor_slices(train_dataset_en)

In [ ]:
# print(train_dataset_en.shape)

In [ ]:
tokenizer_ta= tfds.features.text.SubwordTextEncoder.build_from_corpus((element for element in train_dataset_ta.as_numpy_iterator()), target_vocab_size=10000)

In [ ]:
vocab_size = len(tokenizer_en.word_index) + 3
vocab_tar_size = tokenizer_ta.vocab_size + 3

In [ ]:
train_dataset = tf.data.Dataset.zip((train_dataset_en, train_dataset_ta))
print(train_dataset)

<ZipDataset shapes: ((), ()), types: (tf.string, tf.string)>


In [ ]:
train_sentences = 0
for (batch,(inp,tar)) in enumerate(train_dataset):
    #print(inp)
    #break
    train_sentences += 1
print(train_sentences)

26217


In [ ]:
BATCH_SIZE = 64

def convertToSequence(lang1, lang2):
    st=tokenizer_en.texts_to_sequences([lang1.numpy().decode('utf-8').lower()])
    flat_list = [item for sublist in st for item in sublist]

    lang1 = [len(tokenizer_en.word_index)+1] + flat_list + [len(tokenizer_en.word_index)+2]

    lang2 = [tokenizer_ta.vocab_size+1] + tokenizer_ta.encode(lang2.numpy()) + [tokenizer_ta.vocab_size+2]

    return lang1, lang2

In [ ]:
def toSequence(en, ta):
    result_en, result_ta = tf.py_function(convertToSequence, [en, ta], [tf.int64, tf.int64])
    result_en.set_shape([None])
    result_ta.set_shape([None])

    return result_en, result_ta

In [ ]:
train_dataset_indices = train_dataset.map(toSequence)
padded_train_dataset = train_dataset_indices.padded_batch(BATCH_SIZE,padded_shapes=([None],[None]))
for (batch,(inp,tar)) in enumerate(padded_train_dataset):
    print(inp)
    print(tar)
    break
# print(len(input_tensor),len(input_tensor))

tf.Tensor(
[[339252   6098  13530 ...      0      0      0]
 [339252   6101  13573 ...      0      0      0]
 [339252   3371  24713 ...      0      0      0]
 ...
 [339252   4232   3373 ...      0      0      0]
 [339252   4243    222 ...      0      0      0]
 [339252   3371   4290 ...      0      0      0]], shape=(64, 23), dtype=int64)
tf.Tensor(
[[4410   10    6 ...    0    0    0]
 [4410   42    1 ...    0    0    0]
 [4410   10   26 ...    0    0    0]
 ...
 [4410   19    6 ...    0    0    0]
 [4410   52    1 ...    0    0    0]
 [4410   10   26 ...    0    0    0]], shape=(64, 58), dtype=int64)


In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_matrix, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, 300,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
units = 1024
BATCH_SIZE = 64
encoder = Encoder(vocab_size, embedding_matrix, units, BATCH_SIZE)
print(encoder.trainable_variables)

# sample input
sample_hidden = encoder.initialize_hidden_state()
# for (batch,(inp,tar)) in enumerate(train_dataset_indices):
#     t = inp
    # e = tf.keras.layers.Embedding(vocab_size, 300,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
    # g = tf.keras.layers.GRU(4)
    # print(inp,g([b+for e(inp)]))
    # sample_output, sample_hidden = encoder(inp, sample_hidden)
    # print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
    # print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))
    # break
for (batch,(inp,tar)) in enumerate(padded_train_dataset):
    sample_output, sample_hidden = encoder(inp, sample_hidden)
    print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
    print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))
    break
    # print(t.shape)
# print(batch)

[]
Encoder output shape: (batch size, sequence length, units) (64, 23, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    # self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    # context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    # x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x,initial_state = hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [ ]:
embedding_dim = 256
print(sample_hidden.shape, sample_output.shape)
# decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

# sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
#                                       sample_hidden, sample_output)

# print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

(64, 1024) (64, 23, 1024)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## Checkpoints (Object-based saving)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [ ]:

def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tokenizer_ta.vocab_size+1] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  train_accuracy(targ[:, t], predictions)

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 20
steps_per_epoch = train_sentences//BATCH_SIZE

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(padded_train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f} Accuracy {:.3f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy(),train_accuracy.result()))
  # saving (checkpoint) the model every 2 epochs
#   if (epoch + 1) % 2 == 0:
#     checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.8639 Accuracy 0.000
Epoch 1 Batch 100 Loss 0.8215 Accuracy 0.009
Epoch 1 Batch 200 Loss 0.9243 Accuracy 0.011
Epoch 1 Batch 300 Loss 1.0259 Accuracy 0.012
Epoch 1 Batch 400 Loss 0.8946 Accuracy 0.013
Epoch 1 Loss 0.9809
Time taken for 1 epoch 352.8883934020996 sec

Epoch 2 Batch 0 Loss 1.0417 Accuracy 0.013
Epoch 2 Batch 100 Loss 0.5331 Accuracy 0.013
Epoch 2 Batch 200 Loss 0.6735 Accuracy 0.013
Epoch 2 Batch 300 Loss 0.7786 Accuracy 0.014
Epoch 2 Batch 400 Loss 0.7218 Accuracy 0.014
Epoch 2 Loss 0.6547
Time taken for 1 epoch 352.0446343421936 sec

Epoch 3 Batch 0 Loss 0.8325 Accuracy 0.014
Epoch 3 Batch 100 Loss 0.4357 Accuracy 0.014
Epoch 3 Batch 200 Loss 0.5647 Accuracy 0.014
Epoch 3 Batch 300 Loss 0.6634 Accuracy 0.014
Epoch 3 Batch 400 Loss 0.6332 Accuracy 0.014
Epoch 3 Loss 0.5475
Time taken for 1 epoch 350.7013142108917 sec

Epoch 4 Batch 0 Loss 0.7129 Accuracy 0.014
Epoch 4 Batch 100 Loss 0.3776 Accuracy 0.014
Epoch 4 Batch 200 Loss 0.4939 Accuracy 0.014


In [ ]:
encoder.save_weights('/content/drive/My Drive/encoder_new.h5')

In [ ]:
decoder.save_weights('/content/drive/My Drive/decoder_new.h5')

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [ ]:
def evaluate(sentence,max_length_targ=100):
#   attention_plot = np.zeros((max_length_targ, max_length_inp))

  start_token = [len(tokenizer_en.word_index)+1]
  end_token = [len(tokenizer_en.word_index) + 2]
  
  st=tokenizer_en.texts_to_sequences([sentence.lower()])
  flat_list = [item for sublist in st for item in sublist]
  inp_sentence = start_token + flat_list + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  result = []

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(encoder_input, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer_ta.vocab_size+1] , 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    # attention_weights = tf.reshape(attention_weights, (-1, ))
    # attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    
    result += [predicted_id]
    
    if predicted_id == tokenizer_ta.vocab_size+2:
      return result, sentence
    
    
    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
def translate(sentence, plot=''):
  result,sentence = evaluate(sentence.replace('\'',' \''))
  
  predicted_sentence = tokenizer_ta.decode([i for i in result 
                                            if i <= tokenizer_ta.vocab_size])  
  print(result)
  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(predicted_sentence))

In [ ]:
translate("- NO, HE'S NOT MY DAD.")
print ("நான் ஞாயிறுகளில் அவளை நாம் மற்ற கூறினார்.")

[40, 43, 1, 14, 86, 42, 4, 28, 4, 17, 21, 14, 24, 43, 1, 14, 51, 4411]
Input: - NO, HE 'S NOT MY DAD.
Predicted translation: - இல்லை, அவர் என் வேலை இல்லை.
நான் ஞாயிறுகளில் அவளை நாம் மற்ற கூறினார்.


In [ ]:
encoder.load_weights('/content/drive/My Drive/DEEP LEARNING ASSIGNMENTS/Copy of encoder_new.h5')

In [ ]:
decoder.load_weights('/content/drive/My Drive/DEEP LEARNING ASSIGNMENTS/Copy of decoder_new.h5')

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
